In [1]:
%%capture

!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git 

In [2]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

from huggingface_hub import login 
from transformers import TrainingArguments 
from datasets import load_dataset

import wandb

from kaggle_secrets import UserSecretsClient

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:

user_secrets = UserSecretsClient()
hugging_face_token = user_secrets.get_secret("HF_Token")
wnb_token = user_secrets.get_secret("wnb")


login(hugging_face_token)

wandb.login(key=wnb_token)
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Medical COT Dataset_YouTube Walkthrough', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: manishsharmachd19 (manishsharmachd19-punjab-engineering-college). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  
    max_seq_length=max_seq_length,
    dtype=dtype, 
    load_in_4bit=load_in_4bit, 
    token=hugging_face_token, 
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>{}"""

In [8]:
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or 
              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, 
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda") 


outputs = model.generate(
    input_ids=inputs.input_ids, 
    attention_mask=inputs.attention_mask, 
    max_new_tokens=1200, 
    use_cache=True, 
)


response = tokenizer.batch_decode(outputs)


print(response[0].split("### Response:")[1])  


<think>
Okay, so I need to figure out what cystometry would show for this 61-year-old woman. Let me start by breaking down the information given.

First, she has a history of involuntary urine loss, specifically during activities like coughing or sneezing. That makes me think of stress urinary incontinence. But it's noted that she doesn't leak at night, so maybe it's not purely a night-time issue. She underwent a gynecological exam and a Q-tip test. I'm not entirely sure what the Q-tip test entails, but I think it's used to check for urethral obstruction or to assess how the urethra functions.

Since she's having incontinence during activities that involve coughing, which can increase intra-abdominal pressure, this points towards a possible urethral issue rather than something related to the lower spinal cord or brain, like in spinal cord injury or multiple sclerosis. So, maybe her problem is at the urethral level.

Now, considering cystometry, which is a diagnostic tool used to measu

In [9]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""


In [10]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)
dataset

README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/74.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25371 [00:00<?, ? examples/s]

Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 500
})

In [11]:
dataset[1]

{'Question': 'A 45-year-old man with a history of alcohol use, who has been abstinent for the past 10 years, presents with sudden onset dysarthria, shuffling gait, and intention tremors. Given this clinical presentation and history, what is the most likely diagnosis?',
 'Complex_CoT': "Alright, let’s break this down. We have a 45-year-old man here, who suddenly starts showing some pretty specific symptoms: dysarthria, shuffling gait, and those intention tremors. This suggests something's going wrong with motor control, probably involving the cerebellum or its connections.\n\nNow, what's intriguing is that he's had a history of alcohol use, but he's been off it for the past 10 years. Alcohol can do a number on the cerebellum, leading to degeneration, and apparently, the effects can hang around or even appear long after one stops drinking.\n\nAt first glance, these symptoms look like they could be some kind of chronic degeneration, maybe something like alcoholic cerebellar degeneration, 

In [12]:
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [13]:
def formatting_prompts_func(examples):  
    inputs = examples["Question"]       
    cots = examples["Complex_CoT"]      
    outputs = examples["Response"]
    
    texts = []  
    

    for input, cot, output in zip(inputs, cots, outputs):  
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN 
        texts.append(text)  

    return {
        "text": texts,  
    }

In [17]:
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune["text"][1]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 45-year-old man with a history of alcohol use, who has been abstinent for the past 10 years, presents with sudden onset dysarthria, shuffling gait, and intention tremors. Given this clinical presentation and history, what is the most likely diagnosis?\n\n### Response:\n<think>\nAlright, let’s break this down. We have a 45-year-old man here, who suddenly starts showing some pretty specific symptoms: dysarthria, shuffling gait, and those intention tremors. This suggests something's going wrong w

In [18]:
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16, 
    target_modules=[  
        "q_proj",   
        "k_proj",   
        "v_proj",  
        "o_proj",   
        "gate_proj", 
        "up_proj",   
        "down_proj", 
    ],
    lora_alpha=16, 
    lora_dropout=0, 
    bias="none", 
    use_gradient_checkpointing="unsloth", 
    random_state=3407,  
    use_rslora=False, 
    loftq_config=None, 
)

Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [19]:
trainer = SFTTrainer(
    model=model_lora,  
    tokenizer=tokenizer, 
    train_dataset=dataset_finetune, 
    dataset_text_field="text", 
    max_seq_length=max_seq_length, 
    dataset_num_proc=2, 


    args=TrainingArguments(
        per_device_train_batch_size=2,  
        gradient_accumulation_steps=4, 
        num_train_epochs=1,
        warmup_steps=5, 
        max_steps=60,
        learning_rate=2e-4,  
        fp16=not is_bfloat16_supported(), 
        bf16=is_bfloat16_supported(), 
        logging_steps=10, 
        optim="adamw_8bit", 
        weight_decay=0.01, 
        lr_scheduler_type="linear", 
        seed=3407, 
        output_dir="outputs", 
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

In [20]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.914200
20,1.421000
30,1.366000
40,1.328400
50,1.295100
60,1.326900


In [22]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▄▂▁▁▂
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▁▁▁
total_flos,4.008346716644966e+16
train/epoch,1.896
train/global_step,60
train/grad_norm,0.20182
train/learning_rate,0
train/loss,1.3269


In [21]:
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing 
              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, 
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""


FastLanguageModel.for_inference(model_lora)  


inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")


outputs = model_lora.generate(
    input_ids=inputs.input_ids,         
    attention_mask=inputs.attention_mask, 
    max_new_tokens=1200,                  
    use_cache=True,                        
)


response = tokenizer.batch_decode(outputs)


print(response[0].split("### Response:")[1])


<think>
Alright, let's think about this. We have a 61-year-old woman who's been dealing with involuntary urine loss whenever she coughs or sneezes. That sounds like a classic case of urinary incontinence, probably stress incontinence. Now, stress incontinence usually means the bladder isn't closing properly, so when she coughs, it just pops open and she loses urine.

Now, let's consider what happens in cystometry. This test measures how much urine is left in the bladder and how the detrusor muscle contracts. If she has stress incontinence, the bladder might not be able to hold much urine at all because it's not closing properly. So, when we do the test, we might find that her residual volume is low. That makes sense because if the bladder isn't closing, it doesn't hold much.

Now, about the detrusor contractions. If the bladder can't close, the detrusor might not contract strongly enough to push the urine out. So, during the cystometry, we might not see strong contractions. This is be

In [23]:
question = """A 59-year-old man presents with a fever, chills, night sweats, and generalized fatigue, 
              and is found to have a 12 mm vegetation on the aortic valve. Blood cultures indicate gram-positive, catalase-negative, 
              gamma-hemolytic cocci in chains that do not grow in a 6.5% NaCl medium. 
              What is the most likely predisposing factor for this patient's condition?"""


FastLanguageModel.for_inference(model_lora)  


inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")


outputs = model_lora.generate(
    input_ids=inputs.input_ids,         
    attention_mask=inputs.attention_mask, 
    max_new_tokens=1200,                  
    use_cache=True,                        
)


response = tokenizer.batch_decode(outputs)


print(response[0].split("### Response:")[1])


<think>
Okay, let's see what's going on here. We have a 59-year-old man with symptoms like fever, chills, night sweats, and fatigue. That sounds pretty serious, like a systemic infection. And the key clue is that he's got a vegetation on his aortic valve. That's concerning because it means he might have an infection of the heart valves, which is called endocarditis.

Now, the blood cultures show gram-positive, catalase-negative, gamma-hemolytic cocci in chains. Hmm, I remember that gram-positive, catalase-negative cocci in chains are usually Streptococcus. But wait, Streptococcus doesn't grow in high salt concentrations like 6.5% NaCl. Oh, but these are gram-positive, so I need to think again.

Oh, right! Enterococcus doesn't grow in high salt either. But wait, Enterococcus is gram-positive, catalase-negative, and can be part of the normal flora. But in this case, they're in chains, which is more of a Streptococcus trait. Hmm, this is confusing.

Oh, wait a minute! If these are gram-p